# 02 - Feature Engineering com Dask (Otimizado para Big Data)

Este notebook implementa feature engineering usando **Dask**, uma biblioteca projetada para processar datasets maiores que a RAM disponível.

## Vantagens do Dask:
- ✅ **Processamento Out-of-Core**: Datasets maiores que a RAM
- ✅ **Paralelização Automática**: Usa todos os cores da CPU
- ✅ **API Similar ao Pandas**: Fácil migração
- ✅ **Computação Lazy**: Executa apenas quando necessário
- ✅ **Escalabilidade**: Funciona em clusters distribuídos

In [1]:
import dask.dataframe as dd
import dask.array as da
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar Dask
from dask.distributed import Client, progress
import dask

# Configurar para usar threads (melhor para I/O)
dask.config.set(scheduler='threads')

print('🚀 Dask configurado para processamento de Big Data')
print('💡 Datasets maiores que RAM serão processados automaticamente')

🚀 Dask configurado para processamento de Big Data
💡 Datasets maiores que RAM serão processados automaticamente


## 1. Carregamento com Dask

In [2]:
# Carregar dados com Dask (lazy loading)
print('📂 Carregando dados com Dask...')

# Carregar apenas colunas essenciais para economizar memória
colunas_essenciais = [
    'internal_store_id', 
    'internal_product_id', 
    'transaction_date', 
    'quantity',
    'gross_value',
    'distributor_id'
]

# Dask lê o arquivo sem carregar na memória
transacoes_dask = dd.read_parquet(
    '../data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet',
    columns=colunas_essenciais
)

print(f'📊 Dados carregados (lazy): {transacoes_dask.shape[0].compute():,} registros')
print(f'📊 Colunas: {list(transacoes_dask.columns)}')
print(f'🧠 Memória: Não carregado na RAM ainda (lazy evaluation)')

# Renomear colunas
transacoes_dask = transacoes_dask.rename(columns={
    'internal_store_id': 'pdv_id',
    'internal_product_id': 'produto_id',
    'transaction_date': 'data',
    'quantity': 'quantidade',
    'gross_value': 'valor'
})

# Converter data para datetime
transacoes_dask['data'] = dd.to_datetime(transacoes_dask['data'])

print('✅ Estrutura de dados preparada (lazy)')

📂 Carregando dados com Dask...
📊 Dados carregados (lazy): 6,560,698 registros
📊 Colunas: ['internal_store_id', 'internal_product_id', 'transaction_date', 'quantity', 'gross_value', 'distributor_id']
🧠 Memória: Não carregado na RAM ainda (lazy evaluation)
✅ Estrutura de dados preparada (lazy)


## 2. Criação de Features Temporais com Dask

In [3]:
# Adicionar features temporais (ainda lazy)
print('📅 Criando features temporais...')

# Semana (usando map_partitions para operações customizadas)
def add_week_features(df):
    df = df.copy()
    df['semana'] = df['data'].dt.to_period('W-MON').dt.start_time
    df['mes'] = df['data'].dt.month
    df['semana_ano'] = df['data'].dt.isocalendar().week
    df['ano'] = df['data'].dt.year
    return df

transacoes_dask = transacoes_dask.map_partitions(
    add_week_features, 
    meta=transacoes_dask._meta.assign(
        semana=pd.Timestamp('2022-01-01'),
        mes=1,
        semana_ano=1,
        ano=2022
    )
)

print('✅ Features temporais adicionadas (lazy)')

📅 Criando features temporais...
✅ Features temporais adicionadas (lazy)


## 3. Agregação Semanal com Dask

In [4]:
# Agregação semanal usando Dask (processamento paralelo)
print('🔄 Iniciando agregação semanal com Dask...')

# Dask agrupa e processa em paralelo
agregacao_semanal_dask = transacoes_dask.groupby(['semana', 'pdv_id', 'produto_id']).agg({
    'quantidade': ['sum', 'count'],
    'valor': 'sum',
    'distributor_id': 'first'
})

# Flatten columns
agregacao_semanal_dask.columns = [
    'quantidade', 'num_transacoes', 'valor', 'distributor_id'
]

# Reset index
agregacao_semanal_dask = agregacao_semanal_dask.reset_index()

print('🔄 Agregação configurada (lazy). Executando...')

# EXECUTAR a agregação (aqui que realmente processa)
agregacao_semanal = agregacao_semanal_dask.compute()

print(f'📊 Agregação semanal concluída: {agregacao_semanal.shape}')
print(f'   • Combinações semana/PDV/produto: {len(agregacao_semanal):,}')
print(f'   • PDVs únicos: {agregacao_semanal["pdv_id"].nunique():,}')
print(f'   • Produtos únicos: {agregacao_semanal["produto_id"].nunique():,}')

# Converter de volta para Dask para próximas operações
agregacao_semanal_dask = dd.from_pandas(agregacao_semanal, npartitions=4)

del transacoes_dask  # Liberar memória
print('✅ Dados de transação liberados da memória')

🔄 Iniciando agregação semanal com Dask...
🔄 Agregação configurada (lazy). Executando...
📊 Agregação semanal concluída: (6241315, 7)
   • Combinações semana/PDV/produto: 6,241,315
   • PDVs únicos: 15,086
   • Produtos únicos: 7,092
✅ Dados de transação liberados da memória


## 4. Grid Inteligente com Dask

In [5]:
# Estratégia Grid Inteligente usando Dask
print('🎯 Criando Grid Inteligente com Dask...')

# Obter combinações ativas e semanas únicas
combinacoes_ativas = agregacao_semanal[['pdv_id', 'produto_id']].drop_duplicates()
semanas_unicas = sorted(agregacao_semanal['semana'].unique())

print(f'   • Combinações ativas: {len(combinacoes_ativas):,}')
print(f'   • Semanas: {len(semanas_unicas)}')
print(f'   • Total registros no grid: {len(combinacoes_ativas) * len(semanas_unicas):,}')

# Criar grid usando processamento em lotes otimizado
def create_grid_batch(combo_batch, semanas):
    """Criar grid para um lote de combinações"""
    import pandas as pd
    from itertools import product
    
    # Criar produto cartesiano
    grid_data = []
    for _, row in combo_batch.iterrows():
        for semana in semanas:
            grid_data.append({
                'semana': semana,
                'pdv_id': row['pdv_id'],
                'produto_id': row['produto_id']
            })
    
    return pd.DataFrame(grid_data)

# Processar em lotes usando Dask
batch_size = 5000
grid_parts = []

for i in range(0, len(combinacoes_ativas), batch_size):
    batch = combinacoes_ativas.iloc[i:i+batch_size]
    print(f'   📦 Lote {i//batch_size + 1}: {len(batch)} combinações')
    
    # Criar grid para este lote
    batch_grid = create_grid_batch(batch, semanas_unicas)
    
    # Converter para Dask DataFrame
    batch_grid_dask = dd.from_pandas(batch_grid, npartitions=2)
    
    # Merge com vendas reais
    batch_merged = batch_grid_dask.merge(
        agregacao_semanal_dask,
        on=['semana', 'pdv_id', 'produto_id'],
        how='left'
    )
    
    # Preencher zeros
    batch_merged['quantidade'] = batch_merged['quantidade'].fillna(0)
    batch_merged['valor'] = batch_merged['valor'].fillna(0)
    batch_merged['num_transacoes'] = batch_merged['num_transacoes'].fillna(0)
    
    # Computar e armazenar
    grid_parts.append(batch_merged.compute())

# Concatenar todas as partes
print('🔗 Concatenando grid completo...')
dados_completos = pd.concat(grid_parts, ignore_index=True)

print(f'✅ Grid Inteligente criado: {dados_completos.shape}')
print(f'   • Zeros: {(dados_completos["quantidade"] == 0).sum():,} ({(dados_completos["quantidade"] == 0).mean()*100:.1f}%)')
print(f'   • Não-zeros: {(dados_completos["quantidade"] > 0).sum():,}')

🎯 Criando Grid Inteligente com Dask...
   • Combinações ativas: 1,044,310
   • Semanas: 53
   • Total registros no grid: 55,348,430
   📦 Lote 1: 5000 combinações
   📦 Lote 2: 5000 combinações
   📦 Lote 3: 5000 combinações
   📦 Lote 4: 5000 combinações
   📦 Lote 5: 5000 combinações
   📦 Lote 6: 5000 combinações
   📦 Lote 7: 5000 combinações
   📦 Lote 8: 5000 combinações
   📦 Lote 9: 5000 combinações
   📦 Lote 10: 5000 combinações
   📦 Lote 11: 5000 combinações
   📦 Lote 12: 5000 combinações
   📦 Lote 13: 5000 combinações
   📦 Lote 14: 5000 combinações
   📦 Lote 15: 5000 combinações
   📦 Lote 16: 5000 combinações
   📦 Lote 17: 5000 combinações
   📦 Lote 18: 5000 combinações
   📦 Lote 19: 5000 combinações
   📦 Lote 20: 5000 combinações
   📦 Lote 21: 5000 combinações
   📦 Lote 22: 5000 combinações
   📦 Lote 23: 5000 combinações
   📦 Lote 24: 5000 combinações
   📦 Lote 25: 5000 combinações
   📦 Lote 26: 5000 combinações
   📦 Lote 27: 5000 combinações
   📦 Lote 28: 5000 combinações
   📦 Lote

## 5. Features Avançadas com Dask

In [6]:
# Usando Polars para eficiência máxima com big data
print('🚀 Convertendo para Polars (mais eficiente que pandas para big data)...')
import polars as pl

# Converter para Polars (muito mais eficiente)
dados_polars = pl.from_pandas(dados_completos)

print('🚀 Criando features avançadas com Polars...')

# Ordenar para garantir consistência das features temporais
print('🔄 Ordenando dados por PDV, produto e semana...')
dados_polars = dados_polars.sort(['pdv_id', 'produto_id', 'semana'])

# Features temporais (Polars é muito mais rápido)
print('📅 Criando features temporais...')
dados_polars = dados_polars.with_columns([
    pl.col('semana').dt.month().alias('mes'),
    pl.col('semana').dt.week().alias('semana_ano'),
    (2 * np.pi * pl.col('semana').dt.month() / 12).sin().alias('mes_sin'),
    (2 * np.pi * pl.col('semana').dt.month() / 12).cos().alias('mes_cos')
])

# Features de lag (Polars faz isso de forma super eficiente)
print('⏰ Criando features de lag...')
lag_exprs = []
for lag in [1, 2, 3, 4]:
    print(f'   📋 Lag {lag}...')
    lag_exprs.append(
        pl.col('quantidade').shift(lag).over(['pdv_id', 'produto_id']).alias(f'quantidade_lag_{lag}')
    )

dados_polars = dados_polars.with_columns(lag_exprs)

# Rolling features (Polars tem excelente suporte para rolling windows)
print('📊 Criando rolling features...')
rolling_exprs = [
    pl.col('quantidade').rolling_mean(window_size=4, min_periods=1).over(['pdv_id', 'produto_id']).alias('quantidade_media_4w'),
    pl.col('quantidade').rolling_std(window_size=4, min_periods=1).over(['pdv_id', 'produto_id']).fill_null(0).alias('quantidade_std_4w'),
    pl.col('quantidade').rolling_max(window_size=4, min_periods=1).over(['pdv_id', 'produto_id']).alias('quantidade_max_4w'),
    pl.col('quantidade').rolling_min(window_size=4, min_periods=1).over(['pdv_id', 'produto_id']).alias('quantidade_min_4w'),
]

dados_polars = dados_polars.with_columns(rolling_exprs)

# Features categóricas (usando hash de forma eficiente)
print('🏷️ Criando features categóricas...')
dados_polars = dados_polars.with_columns([
    (pl.col('pdv_id').cast(pl.Utf8).hash() % 100).alias('pdv_hash'),
    (pl.col('produto_id').cast(pl.Utf8).hash() % 100).alias('produto_hash')
])

# Features de interação
print('🔗 Criando features de interação...')
dados_polars = dados_polars.with_columns([
    (pl.col('pdv_hash') * 100 + pl.col('produto_hash')).alias('pdv_produto_hash')
])

# Features estatísticas históricas (Polars é muito eficiente em group_by)
print('📈 Criando features estatísticas históricas...')
stats_historicas = dados_polars.group_by(['pdv_id', 'produto_id']).agg([
    pl.col('quantidade').mean().alias('hist_mean'),
    pl.col('quantidade').std().alias('hist_std'),
    pl.col('quantidade').max().alias('hist_max'),
    pl.col('quantidade').count().alias('hist_count')
])

# Join com dataset principal (Polars otimiza joins automaticamente)
dados_polars = dados_polars.join(
    stats_historicas, 
    on=['pdv_id', 'produto_id'], 
    how='left'
)

print('✅ Features avançadas criadas com Polars!')
print(f'📊 Shape final: {dados_polars.shape}')
print(f'💾 Memória otimizada pelo Polars')

🚀 Convertendo para Polars (mais eficiente que pandas para big data)...
🚀 Criando features avançadas com Polars...
🔄 Ordenando dados por PDV, produto e semana...
📅 Criando features temporais...
⏰ Criando features de lag...
   📋 Lag 1...
   📋 Lag 2...
   📋 Lag 3...
   📋 Lag 4...
📊 Criando rolling features...
🏷️ Criando features categóricas...
🔗 Criando features de interação...
📈 Criando features estatísticas históricas...
✅ Features avançadas criadas com Polars!
📊 Shape final: (55348430, 26)
💾 Memória otimizada pelo Polars


## 6. Execução e Limpeza Final

In [7]:
# Limpeza final com Polars (muito mais eficiente)
print('🧹 Aplicando limpeza final com Polars...')

# Remover registros sem lag_4 (onde lag_4 é null)
dados_limpos = dados_polars.filter(pl.col('quantidade_lag_4').is_not_null())

print(f'✅ Dados limpos: {dados_limpos.shape}')
print(f'   • Período: {dados_limpos.select(pl.col("semana").min()).item()} até {dados_limpos.select(pl.col("semana").max()).item()}')
print(f'   • Semanas: {dados_limpos.select(pl.col("semana").n_unique()).item()}')
print(f'   • Features: {len(dados_limpos.columns)}')

# Verificar memória usada
print(f'🧠 Estimated memory usage: {dados_limpos.estimated_size("mb")} MB')

🧹 Aplicando limpeza final com Polars...
✅ Dados limpos: (51171190, 26)
   • Período: 2022-01-25 00:00:00 até 2022-12-27 00:00:00
   • Semanas: 49
   • Features: 26
🧠 Estimated memory usage: 9974.253155708313 MB


## 7. Salvamento e Metadados

In [8]:
# Salvar dataset final usando Polars (muito mais eficiente)
print('💾 Salvando dataset final com Polars...')

# Polars pode salvar diretamente para CSV e Parquet de forma otimizada
dados_limpos.write_csv('../data/dados_features_completo.csv')
dados_limpos.write_parquet('../data/dados_features_completo.parquet')

print('✅ Dataset salvo em CSV e Parquet')

# Converter uma amostra para pandas para metadados compatíveis
dados_sample = dados_limpos.head(1000).to_pandas()

# Metadados
import pickle

metadata = {
    'data_processamento': pd.Timestamp.now(),
    'total_registros': dados_limpos.shape[0],
    'total_features': len(dados_limpos.columns),
    'combinacoes_pdv_produto': dados_limpos.select([pl.col('pdv_id'), pl.col('produto_id')]).unique().shape[0],
    'semanas_cobertas': dados_limpos.select(pl.col("semana").n_unique()).item(),
    'periodo_treino': f"{dados_limpos.select(pl.col('semana').min()).item()} a {dados_limpos.select(pl.col('semana').max()).item()}",
    'estrategia': 'Grid Inteligente com Dask + Polars - Big Data Optimized',
    'features_criadas': dados_limpos.columns,
    'tecnologia': 'Dask + Polars for Maximum Performance',
    'memoria_otimizada': f'{dados_limpos.estimated_size("mb")} MB'
}

with open('../data/feature_engineering_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print('📋 Metadados salvos')

# Estatísticas finais usando Polars (muito mais rápido)
print('\n🎉 FEATURE ENGINEERING COM DASK + POLARS CONCLUÍDO!')
print('=' * 60)
print(f'📊 Dataset final: {dados_limpos.shape}')
print(f'💾 Arquivos salvos:')
print('   • dados_features_completo.csv')
print('   • dados_features_completo.parquet')  
print('   • feature_engineering_metadata.pkl')
print(f'\n🏷️ Features principais:')
features_importantes = ['quantidade', 'quantidade_lag_1', 'quantidade_lag_2', 
                       'quantidade_lag_4', 'quantidade_media_4w', 'mes_sin', 'mes_cos']
for feat in features_importantes:
    if feat in dados_limpos.columns:
        print(f'   ✅ {feat}')

print('\n🚀 Pronto para Modelagem com dados super-otimizados!')
print(f'📈 Distribuição target:')

# Calcular estatísticas com Polars (muito mais eficiente)
zeros = dados_limpos.select((pl.col("quantidade") == 0).sum()).item()
total = dados_limpos.shape[0]
nao_zeros = total - zeros

print(f'   • Zeros: {zeros:,} ({zeros/total*100:.1f}%)')
print(f'   • Não-zeros: {nao_zeros:,} ({nao_zeros/total*100:.1f}%)')
print(f'🧠 Memória total usada: {dados_limpos.estimated_size("mb")} MB')
print('💡 Polars otimizou automaticamente o uso de memória!')

💾 Salvando dataset final com Polars...
✅ Dataset salvo em CSV e Parquet
📋 Metadados salvos

🎉 FEATURE ENGINEERING COM DASK + POLARS CONCLUÍDO!
📊 Dataset final: (51171190, 26)
💾 Arquivos salvos:
   • dados_features_completo.csv
   • dados_features_completo.parquet
   • feature_engineering_metadata.pkl

🏷️ Features principais:
   ✅ quantidade
   ✅ quantidade_lag_1
   ✅ quantidade_lag_2
   ✅ quantidade_lag_4
   ✅ quantidade_media_4w
   ✅ mes_sin
   ✅ mes_cos

🚀 Pronto para Modelagem com dados super-otimizados!
📈 Distribuição target:
   • Zeros: 45,254,351 (88.4%)
   • Não-zeros: 5,916,839 (11.6%)
🧠 Memória total usada: 9974.253155708313 MB
💡 Polars otimizou automaticamente o uso de memória!
